<a href="https://colab.research.google.com/github/kla55/transformer/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load tokenizer and model (e.g., BERT for text classification)
model_name = "bert-base-uncased"
# The AutoTokenizer.from_pretrained method in the Hugging Face Transformers library is a powerful tool for loading pre-trained tokenizers.
# This method simplifies the process of converting text data into a format that can be fed into models for various NLP tasks.
tokenizer = AutoTokenizer.from_pretrained(model_name)
# a method from the Hugging Face Transformers library used to load a pre-trained transformer model specifically designed
# for sequence classification tasks (e.g., sentiment analysis, spam detection, or topic classification).
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("imdb")  # Example: IMDB movie reviews

# Tokenize the text data
# tokenizer(examples["text"]) converts the input text into numerical token IDs.
# The tokenizer returns a dictionary with:
# input_ids: The list of token IDs representing the text.
# attention_mask: A binary mask indicating which tokens are meaningful (1) and which are padding tokens (0). The model uses this to ignore padding during computation.
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
# Train-test split
train_dataset = tokenized_dataset["train"]
valid_dataset = tokenized_dataset["test"]

In [26]:
# Convert datasets into PyTorch DataLoader for batching and shuffling.

from torch.utils.data import DataLoader

batch_size = 16
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)

labels = torch.as_tensor([lbl for lbl in labels])

In [17]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

In [6]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
import torch
from torch.nn import CrossEntropyLoss
from torch.nn.functional import softmax

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(model.to(device))

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
token_ids = [101, 101, 101, 101]  # Example token IDs
decoded_text = tokenizer.decode(token_ids)
print(decoded_text)

[CLS] [CLS] [CLS] [CLS]


In [32]:
num_epochs = 3
criterion = CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Move tensors to the appropriate device
        inputs = {key: val.to(device) for key, val in batch.items() if key in ["input_ids", "attention_mask"]}
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} completed.")


AttributeError: 'list' object has no attribute 'to'

In [33]:
num_epochs = 3
criterion = CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Move tensors to the appropriate device
        inputs = {}
        for key, val in batch.items():
            if key in ["input_ids", "attention_mask"]:
                # Convert lists to tensors if necessary and move to device
                if isinstance(val, list):
                    val = torch.tensor(val)
                inputs[key] = val.to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} completed.")


TypeError: only integer tensors of a single element can be converted to an index

In [34]:
num_epochs = 3
criterion = CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Move tensors to the appropriate device
        inputs = {}
        for key, val in batch.items():
            if key in ["input_ids", "attention_mask"]:
                # Convert lists to tensors if necessary and move to device
                if isinstance(val, list):
                    val = torch.tensor(val)
                inputs[key] = val.to(dtype=torch.long, device=device)
        # Convert labels to LongTensor and move to device
        labels = batch["label"]
        if isinstance(labels, list):
            labels = torch.tensor(labels)
        labels = labels.to(device).long()

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} completed.")


TypeError: only integer tensors of a single element can be converted to an index

In [31]:
num_epochs = 3
criterion = CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        inputs = {key: val.to(device) for key, val in batch.items() if key in ["input_ids", "attention_mask"]}

        # inputs = {}
        # for key, val in batch.items():
            # if key in ["input_ids", "attention_mask"]:
            #     # Move existing tensors to device instead of creating new ones
            #     inputs[key] = torch.tensor(val).to(device)

        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} completed.")

AttributeError: 'list' object has no attribute 'to'

In [29]:
num_epochs = 3
critierion = CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for bath


AttributeError: 'list' object has no attribute 'to'

In [35]:
import torch
from torch.nn import CrossEntropyLoss
from torch.nn.functional import softmax

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(model.to(device))

num_epochs = 3
criterion = CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Create inputs dictionary
        inputs = {}
        for key, val in batch.items():
            if key in ["input_ids", "attention_mask"]:
                # Convert lists to PyTorch tensors and move to device
                inputs[key] = torch.tensor(val).to(dtype=torch.long, device=device)

        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} completed.")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

TypeError: only integer tensors of a single element can be converted to an index

In [10]:
import torch
from torch.nn import CrossEntropyLoss
from torch.nn.functional import softmax

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(model.to(device))

num_epochs = 3
criterion = CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Create inputs dictionary
        inputs = {}
        for key, val in batch.items():
            if key in ["input_ids", "attention_mask"]:
                # Convert lists to PyTorch tensors and move to device
                inputs[key] = torch.tensor(val).to(device)

        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} completed.")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

TypeError: only integer tensors of a single element can be converted to an index

In [11]:
import torch
from torch.nn import CrossEntropyLoss
from torch.nn.functional import softmax

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(model.to(device))

num_epochs = 3
criterion = CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Create inputs dictionary

SyntaxError: incomplete input (<ipython-input-11-c5cee7e10995>, line 14)

In [12]:
import torch
from torch.nn import CrossEntropyLoss
from torch.nn.functional import softmax

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(model.to(device))

num_epochs = 3
criterion = CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Create inputs dictionary
        inputs = {}
        for key, val in batch.items():
            if key in ["input_ids", "attention_mask"]:
                # Move existing tensors to device instead of creating new ones
                inputs[key] = val.to(device)

        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} completed.")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

AttributeError: 'list' object has no attribute 'to'

In [13]:
import torch
from torch.nn import CrossEntropyLoss
from torch.nn.functional import softmax

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(model.to(device))

num_epochs = 3
criterion = CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        # Create inputs dictionary
        inputs = {}
        for key, val in batch.items():
            if key in ["input_ids", "attention_mask"]:
                # Convert lists to PyTorch tensors and move to device
                # The values of 'input_ids' and 'attention_mask' in the 'batch' dictionary are lists.
                # We need to convert these lists to PyTorch tensors before moving them to the device.
                inputs[key] = torch.tensor(val).to(device)

        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} completed.")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
import torch
from torch.nn import CrossEntropyLoss
from torch.nn.functional import softmax

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
criterion = CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        inputs = {key: val.to(device) for key, val in batch.items() if key in ["input_ids", "attention_mask"]}
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} completed.")


In [37]:
from transformers import AutoTokenizer

# Use a pre-trained tokenizer (e.g., DistilBERT)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [38]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [39]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=16, shuffle=True)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=16)


In [41]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [43]:
from torch.optim import AdamW
import torch

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Use CrossEntropyLoss for binary classification
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
from tqdm import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Training loop
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()
    train_loss = 0

    for batch in tqdm(train_dataloader):
        # Move batch to device
        inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(**inputs)
        loss = loss_fn(outputs.logits, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_dataloader)
    print(f"Training Loss: {avg_train_loss:.4f}")


Epoch 1/3


  6%|▌         | 92/1563 [01:06<17:40,  1.39it/s]

In [ ]:
from sklearn.metrics import accuracy_score

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
        labels = batch["labels"].to(device)

        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "Positive" if prediction == 1 else "Negative"

# Example prediction
print(predict("The movie was fantastic!"))